In [1]:
import qualified Data.HashMap.Strict as H
import System.Random
import Control.Concurrent.Async

content <- map words . lines <$> readFile "kargerMinCut.txt"

choose ls = do
    idx <- randomRIO (0, length ls - 1)
    return $ ls !! idx

chooseOther one ls = do
    other <- choose ls
    if other == one then chooseOther one ls else return (one, other)

In [2]:
import Data.List (foldl')

add adj (x:xs) = H.insert x xs adj

adj = foldl' add H.empty content

In [15]:
contract adj a b = let
    edgesA  = adj H.! a
    edgesB  = adj H.! b
    -- Remove self loops
    edgesAB = filter (\e -> e `notElem` [a, b]) $ edgesA ++ edgesB
    -- New vertex name
    newV    = a ++ (',':b)
    -- Remove old vertices
    adj'    = H.delete a (H.delete b adj)
    -- Add new vertex
    adj''   = H.insert newV edgesAB adj'
    -- Rewrite all occurrences of a b to point to newV
    in map (\e -> if e `elem` [a, b] then newV else e) <$> adj''
    
karger adj 
    | H.size adj == 2 = return adj
    | otherwise = do
        let keys = H.keys adj
        one  <- choose keys
        pair <- chooseOther one (adj H.! one)
        let adj' = uncurry (contract adj) pair
        karger adj'

In [16]:
multiple n adj = replicateConcurrently n (length . head . H.elems <$> karger adj)

minimum <$> multiple 50 adj

17

In [17]:
minimum <$> multiple 100 adj

17

In [18]:
minimum <$> multiple 100 adj

17

In [20]:
minimum <$> multiple 50 adj

17

In [ ]:
minimum <$> multiple 100 adj

In [ ]:
minimum <$> multiple 100 adj